<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/riva_asr_asr-python-advanced-finetune-am-citrinet-tao-deployment/nvidia_logo.png" style="width: 90px; float: right;">

# How to deploy custom Acoustic Model (Citrinet) trained with TAO Toolkit on Riva
This tutorial walks you through the deployment of custom acoustic model (Citrinet) trained with TAO Toolkit on Riva.

---
## Riva ServiceMaker
Riva ServiceMaker is a set of tools that aggregates all the necessary artifacts (models, files, configurations, and user settings) for Riva deployment to a target environment. It has two main components:

### Riva-build

This step helps build a Riva-ready version of the model. It’s only output is an intermediate format (called an RMIR) of an end-to-end pipeline for the supported services within Riva. Let's consider an ASR Citrinet model. <br>

`riva-build` is responsible for the combination of one or more exported models (`.riva` files) into a single file containing an intermediate format called Riva Model Intermediate Representation (`.rmir`). This file contains a deployment-agnostic specification of the whole end-to-end pipeline along with all the assets required for the final deployment and inference. For more information, refer to the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/service-asr.html#pipeline-configuration).

In [ ]:
# IMPORTANT: UPDATE MODEL_LOC with `asr-model.riva` ABSOLUTE PATH 
import os
# ServiceMaker Docker
RIVA_SM_CONTAINER = "nvcr.io/nvidia/riva/riva-speech:2.2.1-servicemaker"

# Directory where the .riva model is stored $MODEL_LOC/*.riva
WORKING_DIR = os.path.join(os.getcwd(), "asr_am_finetuning")
MODEL_LOC = WORKING_DIR + "/results/citrinet/riva/"

# Name of the .riva file
MODEL_NAME = "asr-model.riva"

# Key that model is encrypted with, while exporting with TAO
KEY = "tlt_encode"

In [ ]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
! docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
            riva-build speech_recognition /data/asr.rmir:$KEY /data/$MODEL_NAME:$KEY --offline \
            --chunk_size=1.6 \
            --padding_size=1.6 \
            --ms_per_timestep=80 \
            --greedy_decoder.asr_model_delay=-1 \
            --featurizer.use_utterance_norm_params=False \
            --featurizer.precalc_norm_time_steps=0 \
            --featurizer.precalc_norm_params=False \
            --decoder_type=greedy

### Riva-deploy

The deployment tool takes as input one or more Riva Model Intermediate Representation (RMIR) files and a target model repository directory. It creates an ensemble configuration specifying the pipeline for the execution and finally writes all those assets to the output model repository directory.

Be patient! This step would take some time.

In [ ]:
# Syntax: riva-deploy -f dir-for-rmir/model.rmir:key output-dir-for-repository
! docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
            riva-deploy -f  /data/asr.rmir:$KEY /data/models/

---
## Start the Riva Server
After the model repository is generated, we are ready to start the Riva server. First, download the Riva Quick Start resource from NGC. 
Set the path to the directory here:

In [ ]:
! ngc registry resource download-version "nvidia/riva/riva_quickstart:2.2.1"

In [ ]:
# Set the Riva Quick Start directory
RIVA_DIR = os.path.join(os.getcwd(), "riva_quickstart_v2.2.1")

Next, we modify the `config.sh` file to enable relevant Riva services (ASR for the Citrinet model), provide the encryption key, and path to the model repository (`riva_model_loc`) generated in the previous step among other configurations. 

For example, if above the model repository is generated at `$MODEL_LOC/models`, then you can specify `riva_model_loc` as the same directory as `MODEL_LOC`. <br>

Pretrained versions of models specified in `models_asr/nlp/tts` are fetched from NGC. Since we are using our custom model, we can comment it in `models_asr` (and any others that are not relevant to your use case). <br>

#### config.sh snippet
```
# Enable or Disable Riva Services 
service_enabled_asr=true                                                      ## MAKE CHANGES HERE
service_enabled_nlp=false                                                      ## MAKE CHANGES HERE
service_enabled_tts=false                                                     ## MAKE CHANGES HERE

# Specify one or more GPUs to use
# specifying more than one GPU is currently an experimental feature, and may result in undefined behaviours.
gpus_to_use="device=0"

# Specify the encryption key to use to deploy models
MODEL_DEPLOY_KEY="tlt_encode"

# Locations to use for storing models artifacts
#
# If an absolute path is specified, the data will be written to that location
# Otherwise, a docker volume will be used (default).
#
# riva_init.sh will create a `rmir` and `models` directory in the volume or
# path specified. 
#
# RMIR ($riva_model_loc/rmir)
# Riva uses an intermediate representation (RMIR) for models
# that are ready to deploy but not yet fully optimized for deployment. Pretrained
# versions can be obtained from NGC (by specifying NGC models below) and will be
# downloaded to $riva_model_loc/rmir by `riva_init.sh`
# 
# Custom models produced by NeMo or TAO and prepared using riva-build
# may also be copied manually to this location $(riva_model_loc/rmir).
#
# Models ($riva_model_loc/models)
# During the riva_init process, the RMIR files in $riva_model_loc/rmir
# are inspected and optimized for deployment. The optimized versions are
# stored in $riva_model_loc/models. The riva server exclusively uses these
# optimized versions.
riva_model_loc="<add path>"                              ## MAKE CHANGES HERE (Replace with MODEL_LOC)                      
```

**Make sure to do the following before moving forward:**
1. In the file navigator in Jupyter Lab, navigate to riva_quickstart_v2.* and open config.sh
2. Configure settings as shown in the snippet above
   - Set nlp and tts services to false
   - Configure the riva_model_loc path to where the models resulting from riva-deploy are stored

In [ ]:
# set `riva-model-loc` to where the models resulting from riva-deploy are stored. In our case it is MODEL_LOC
!echo $MODEL_LOC

In [ ]:
# Ensure you have permission to execute these scripts
! cd $RIVA_DIR && chmod +x ./riva_stop.sh && chmod +x ./riva_start.sh

In [ ]:
# Stop existing Riva deployments. 
! cd $RIVA_DIR && ./riva_stop.sh config.sh 
# Run Riva Start. This will deploy your model(s).
! cd $RIVA_DIR && ./riva_start.sh config.sh

---
## Run Inference
Once the Riva server is up-and-running with your models, you can send inference requests querying the server. 


### Connect to the Riva Server and Run Inference
Now we can actually query the Riva server. The following cell queries the Riva server (using gRPC) to yield a result.

In [ ]:
import argparse
import grpc
import time
try:
    import riva_api.riva_audio_pb2 as ra # RIVA 2.0.0 and above
except:
    import riva_api.audio_pb2 as ra
import riva_api.riva_asr_pb2 as rasr
import riva_api.riva_asr_pb2_grpc as rasr_srv
import wave

audio_file = "audio_samples/en-US_sample.wav"
server = "localhost:50051"

wf = wave.open(audio_file, 'rb')
with open(audio_file, 'rb') as fh:
    data = fh.read()

channel = grpc.insecure_channel(server)
client = rasr_srv.RivaSpeechRecognitionStub(channel)
config = rasr.RecognitionConfig(
    encoding=ra.AudioEncoding.LINEAR_PCM,
    sample_rate_hertz=wf.getframerate(),
    language_code="en-US",
    max_alternatives=1,
    enable_automatic_punctuation=False,
    audio_channel_count=1
)

request = rasr.RecognizeRequest(config=config, audio=data)

response = client.Recognize(request)
print(response)

---
### Cleanup

You can stop all Docker containers before shutting down the Jupyter kernel. **Caution: The following command will stop all running containers.**

In [ ]:
! docker stop $(docker ps -a -q)